In [ ]:
pip install langchain

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

## langchain OpenAI LLM

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm = OpenAI()
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)

## langchain ChatOpenAI 

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [ ]:
chat = ChatOpenAI(temperature=0.25)

In [ ]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
chat(messages)

### langchain ChatOpenAI (with templates)

In [ ]:
template = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [ ]:
# get a chat completion from the formatted messages
chat(
    chat_prompt.format_prompt(
        input_language="English", output_language="French", text="I love programming."
    ).to_messages()
)

## langchain tool calling python function

In [ ]:
import json

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
tools = [
    Tool(
        name = "GetCurrentWeather",
        func=get_current_weather,
        description="useful for when you need to answer questions about current weather for a given location"
    )
]
mrkl = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [ ]:
mrkl.run("What's the weather like in Boston?")

## langchain calling CCCS tools: AssemblyLine and Beaver

In [ ]:
AL_API_KEY = os.getenv('AL_API_KEY')
AL_USER = os.getenv('AL_USER')
BEAVER_API_KEY = os.getenv('BEAVER_API_KEY')

In [ ]:
from assemblyline_client import get_client
al_client = get_client("https://malware.cyber.gc.ca:443", apikey=(AL_USER, AL_API_KEY))

In [ ]:
import requests
def beaver_search(sha256):
    response = requests.get(
        f"https://beaver.ops.cyber.gc.ca/auth/api/sha256/{sha256}/json",
        headers={
            "X-API-Key": BEAVER_API_KEY,
            "Accept": "application/json",
        }
    )
    return response

In [ ]:
def get_antivirus_service_results_from_al(sha256):
    """Get the AssemblyLine antivirus report for the given sha256"""
    search_result = al_client.search.result(f'sha256:{sha256} AND response.service_name:AntiVirus', fl="*")
    # return "\n".join([i['title_text'] for i in search_result['items'][0]['result']['sections']])
    if len(search_result['items']) > 0:
        return json.dumps([i['title_text'] for i in search_result['items'][0]['result']['sections']])
    else:
        return "No antivirus report available for this sha256 in AssemblyLine"

def get_antivirus_service_results_from_beaver(sha256):
    response = beaver_search(sha256)
    if response.status_code == 200:
        json_data = json.loads(response.text)
        return json.dumps(json_data['malwareReport']['reports']['SHADOW_SERVER'][0]['results'])
    else:
        return "No antivirus report available for this sha256 in Beaver"

In [ ]:
tools = [
    Tool(
        name = "GetAntivirusServiceResultsFromAssemblyLine",
        func=get_antivirus_service_results_from_al,
        description="useful for when you need to answer questions about AssemblyLine antivirus report for a given sha256"
    ),
    Tool(
        name = "GetAntivirusServiceResultsFromBeaver",
        func=get_antivirus_service_results_from_beaver,
        description="useful for when you need to answer questions about Beaver antivirus report for a given sha256"
    )
]
mrkl = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [ ]:
# from langchain.globals import set_verbose, set_debug
# set_verbose(True)
# set_debug(False)

print(mrkl.run("What are the antivirus results for sha256 ca6a054154acb559a495cfea1923933a65c94a325834a3cf9f46d85d3adaacf5?"))